In [1]:
import sqlalchemy
import pandas as pd
from geopy.geocoders import MapBox
from geopy.distance import geodesic

In [2]:
engine = sqlalchemy.create_engine('sqlite:///db/ncaa_history.sqlite')
connection = engine.connect()

## Import SQL Database Tables

In [3]:
games_df = pd.read_sql_table('games_all', engine)
games_df.head()

,location,loser,loser_score,loser_seed,round,winner,winner_score,winner_seed,year
0,"Pittsburgh, PA",Radford,61,16,1,Villanova,87,1,2018
1,"Pittsburgh, PA",Virginia Tech,83,8,1,Alabama,86,9,2018
2,"San Diego, CA",Murray State,68,12,1,West Virginia,85,5,2018
3,"San Diego, CA",Wichita State,75,4,1,Marshall,81,13,2018
4,"Dallas, TX",St. Bonaventure,62,11,1,Florida,77,6,2018


In [4]:
games_df = games_df[['year', 'round', 'location', 'winner', 'winner_seed', 'winner_score', 'loser', 'loser_seed', 'loser_score']]
games_df.head()

,year,round,location,winner,winner_seed,winner_score,loser,loser_seed,loser_score
0,2018,1,"Pittsburgh, PA",Villanova,1,87,Radford,16,61
1,2018,1,"Pittsburgh, PA",Alabama,9,86,Virginia Tech,8,83
2,2018,1,"San Diego, CA",West Virginia,5,85,Murray State,12,68
3,2018,1,"San Diego, CA",Marshall,13,81,Wichita State,4,75
4,2018,1,"Dallas, TX",Florida,6,77,St. Bonaventure,11,62


In [5]:
schools_df = pd.read_sql_table('schools', engine)
schools_df.head()

,location,logo,name,url
0,"Abilene, Texas",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Abilene Christian Wildcats,https://www.sports-reference.com/cbb/schools/a...
1,"USAF Academy, Colorado",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Air Force,https://www.sports-reference.com/cbb/schools/a...
2,"Akron, Ohio",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Akron,https://www.sports-reference.com/cbb/schools/a...
3,"Normal, Alabama",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Alabama A&M Bulldogs,https://www.sports-reference.com/cbb/schools/a...
4,"Tuscaloosa, Alabama",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Alabama,https://www.sports-reference.com/cbb/schools/a...


In [6]:
school_locations = schools_df.loc[:,['location','name']]
school_locations.head()

,location,name
0,"Abilene, Texas",Abilene Christian Wildcats
1,"USAF Academy, Colorado",Air Force
2,"Akron, Ohio",Akron
3,"Normal, Alabama",Alabama A&M Bulldogs
4,"Tuscaloosa, Alabama",Alabama


In [7]:
teams_df = pd.read_sql_table('teams_all', engine)
teams_df.head()

,name,opp_g,opp_mp,opp_opp_ast,opp_opp_blk,opp_opp_drb,opp_opp_fg,opp_opp_fg2,opp_opp_fg2_pct,opp_opp_fg2a,...,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_tov,team_rank_trb,team_stl,team_tov,team_trb,url,year
0,Villanova,40,8075,532,105,928,1028,756,0.49,1543,...,1,1,28,182,8,259,426,1436,https://www.sports-reference.com/cbb/schools/v...,2018
1,Radford,36,7325,413,120,818,819,558,0.484,1152,...,169,312,97,223,93,226,441,1244,https://www.sports-reference.com/cbb/schools/r...,2018
2,Virginia Tech,33,6675,460,124,784,844,552,0.492,1123,...,75,46,213,92,262,191,395,1078,https://www.sports-reference.com/cbb/schools/v...,2018
3,Alabama,36,7200,427,134,863,877,615,0.469,1310,...,90,217,90,346,61,228,513,1290,https://www.sports-reference.com/cbb/schools/a...,2018
4,West Virginia,37,7425,462,148,893,835,547,0.454,1204,...,13,45,5,161,13,301,420,1401,https://www.sports-reference.com/cbb/schools/w...,2018


## School/Game Locations

In [8]:
# merge games table and school locations to start games_stats table
game_stats = games_df.merge(school_locations, how='left', left_on='winner', right_on='name', suffixes = ('','_winner'))
game_stats = game_stats.merge(school_locations, how='left', left_on='loser', right_on='name', suffixes = ('','_loser'))
del game_stats['name']
del game_stats['name_loser']
game_stats.head()

,year,round,location,winner,winner_seed,winner_score,loser,loser_seed,loser_score,location_winner,location_loser
0,2018,1,"Pittsburgh, PA",Villanova,1,87,Radford,16,61,"Villanova, Pennsylvania","Radford, Virginia"
1,2018,1,"Pittsburgh, PA",Alabama,9,86,Virginia Tech,8,83,"Tuscaloosa, Alabama","Blacksburg, Virginia"
2,2018,1,"San Diego, CA",West Virginia,5,85,Murray State,12,68,"Morgantown, West Virginia","Murray, Kentucky"
3,2018,1,"San Diego, CA",Marshall,13,81,Wichita State,4,75,"Huntington, West Virginia","Wichita, Kansas"
4,2018,1,"Dallas, TX",Florida,6,77,St. Bonaventure,11,62,"Gainesville, Florida","St. Bonaventure, New York"


In [9]:
geolocator = MapBox(api_key="pk.eyJ1Ijoib2JuaWNob2xzb24iLCJhIjoiY2pwcHBrbmIxMGdhMTN4cWZ2czR6NDVwcCJ9.T5lnDc1uaxKgp4S18rFyBw", timeout=None)

In [10]:
def get_latlons(column):
    counter = 0
    locations = []
    latlons = []

    for location in column:
        geolocation = geolocator.geocode(location)
        locations.append(geolocation)
        counter +=1
        if (counter % 100 == 0):
            print(counter)

    for location in locations:
        latlon = (location.latitude, location.longitude)
        latlons.append(latlon)
    
    return latlons

In [11]:
game_locations = get_latlons(game_stats['location'])
winner_locations = get_latlons(game_stats['location_winner'])
loser_locations = get_latlons(game_stats['location_loser'])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800


In [12]:
def get_travel_distances(school_location_list):
    travel_distances = []
    
    for index, school_location in enumerate(school_location_list):
        game_location = game_locations[index]
        travel_distance = geodesic(school_location, game_location).miles
        travel_distances.append(travel_distance)
        
    return travel_distances

In [13]:
winner_travel_distances = get_travel_distances(winner_locations)
loser_travel_distances = get_travel_distances(loser_locations)

In [14]:
pd.options.mode.chained_assignment = None

game_stats['travel_miles_winner'] = winner_travel_distances
game_stats['travel_miles_loser'] = loser_travel_distances
game_stats.head()

,year,round,location,winner,winner_seed,winner_score,loser,loser_seed,loser_score,location_winner,location_loser,travel_miles_winner,travel_miles_loser
0,2018,1,"Pittsburgh, PA",Villanova,1,87,Radford,16,61,"Villanova, Pennsylvania","Radford, Virginia",246.946684,230.496683
1,2018,1,"Pittsburgh, PA",Alabama,9,86,Virginia Tech,8,83,"Tuscaloosa, Alabama","Blacksburg, Virginia",645.478758,222.738124
2,2018,1,"San Diego, CA",West Virginia,5,85,Murray State,12,68,"Morgantown, West Virginia","Murray, Kentucky",2117.874565,1658.405787
3,2018,1,"San Diego, CA",Marshall,13,81,Wichita State,4,75,"Huntington, West Virginia","Wichita, Kansas",1982.540076,1170.375993
4,2018,1,"Dallas, TX",Florida,6,77,St. Bonaventure,11,62,"Gainesville, Florida","St. Bonaventure, New York",882.710411,1190.429282


## Create Losing/Winning Records with Season Stats

In [15]:
winners_df = game_stats.rename(columns={'winner':'team_name', 'winner_seed':'team_seed', 'winner_score':'team_score', 
                                       'loser':'opponent_name', 'loser_seed':'opponent_seed', 'loser_score':'opponent_score', 
                                       'location_winner':'team_location', 'location_loser':'opponent_location', 
                                       'travel_miles_winner':'team_travel_miles', 'travel_miles_loser':'opponent_travel_miles'})
winners_df['outcome'] = 'WIN'
winners_df = winners_df[['year', 'round', 'location', 'team_name', 'team_seed', 'team_score', 'team_location', 
                         'team_travel_miles', 'opponent_name', 'opponent_seed', 'opponent_score', 'opponent_location', 
                         'opponent_travel_miles', 'outcome']]
winners_df.head()

,year,round,location,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,outcome
0,2018,1,"Pittsburgh, PA",Villanova,1,87,"Villanova, Pennsylvania",246.946684,Radford,16,61,"Radford, Virginia",230.496683,WIN
1,2018,1,"Pittsburgh, PA",Alabama,9,86,"Tuscaloosa, Alabama",645.478758,Virginia Tech,8,83,"Blacksburg, Virginia",222.738124,WIN
2,2018,1,"San Diego, CA",West Virginia,5,85,"Morgantown, West Virginia",2117.874565,Murray State,12,68,"Murray, Kentucky",1658.405787,WIN
3,2018,1,"San Diego, CA",Marshall,13,81,"Huntington, West Virginia",1982.540076,Wichita State,4,75,"Wichita, Kansas",1170.375993,WIN
4,2018,1,"Dallas, TX",Florida,6,77,"Gainesville, Florida",882.710411,St. Bonaventure,11,62,"St. Bonaventure, New York",1190.429282,WIN


In [16]:
losers_df = game_stats.rename(columns={'winner':'opponent_name', 'winner_seed':'opponent_seed', 'winner_score':'opponent_score', 
                                       'loser':'team_name', 'loser_seed':'team_seed', 'loser_score':'team_score', 
                                       'location_winner':'opponent_location', 'location_loser':'team_location', 
                                       'travel_miles_winner':'opponent_travel_miles', 'travel_miles_loser':'team_travel_miles'})
losers_df['outcome'] = 'LOSS'
losers_df = losers_df[['year', 'round', 'location', 'team_name', 'team_seed', 'team_score', 'team_location', 
                         'team_travel_miles', 'opponent_name', 'opponent_seed', 'opponent_score', 'opponent_location', 
                         'opponent_travel_miles', 'outcome']]
losers_df.head()

,year,round,location,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,outcome
0,2018,1,"Pittsburgh, PA",Radford,16,61,"Radford, Virginia",230.496683,Villanova,1,87,"Villanova, Pennsylvania",246.946684,LOSS
1,2018,1,"Pittsburgh, PA",Virginia Tech,8,83,"Blacksburg, Virginia",222.738124,Alabama,9,86,"Tuscaloosa, Alabama",645.478758,LOSS
2,2018,1,"San Diego, CA",Murray State,12,68,"Murray, Kentucky",1658.405787,West Virginia,5,85,"Morgantown, West Virginia",2117.874565,LOSS
3,2018,1,"San Diego, CA",Wichita State,4,75,"Wichita, Kansas",1170.375993,Marshall,13,81,"Huntington, West Virginia",1982.540076,LOSS
4,2018,1,"Dallas, TX",St. Bonaventure,11,62,"St. Bonaventure, New York",1190.429282,Florida,6,77,"Gainesville, Florida",882.710411,LOSS


In [17]:
dataframes = [winners_df, losers_df]
outcomes_df = pd.concat(dataframes, sort=False)
print(len(winners_df), len(losers_df), len(outcomes_df))

1872 1872 3744


In [18]:
outcomes_incl_team_stats = outcomes_df.merge(teams_df, how='left', left_on=['team_name','year'], right_on=['name','year'], suffixes=('','_stats'))
outcomes_full_stats = outcomes_incl_team_stats.merge(teams_df.add_prefix('opponent_'), how='left', left_on=['opponent_name','year'], right_on=['opponent_name','opponent_year'], suffixes=('','_stats'))
outcomes_full_stats.head()

,year,round,location,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,...,opponent_team_rank_pts,opponent_team_rank_pts_per_g,opponent_team_rank_stl,opponent_team_rank_tov,opponent_team_rank_trb,opponent_team_stl,opponent_team_tov,opponent_team_trb,opponent_url,opponent_year
0,2018,1,"Pittsburgh, PA",Villanova,1,87,"Villanova, Pennsylvania",246.946684,Radford,16,...,169,312,97,223,93,226,441,1244,https://www.sports-reference.com/cbb/schools/r...,2018
1,2018,1,"Pittsburgh, PA",Alabama,9,86,"Tuscaloosa, Alabama",645.478758,Virginia Tech,8,...,75,46,213,92,262,191,395,1078,https://www.sports-reference.com/cbb/schools/v...,2018
2,2018,1,"San Diego, CA",West Virginia,5,85,"Morgantown, West Virginia",2117.874565,Murray State,12,...,126,61,129,74,143,216,384,1186,https://www.sports-reference.com/cbb/schools/m...,2018
3,2018,1,"San Diego, CA",Marshall,13,81,"Huntington, West Virginia",1982.540076,Wichita State,4,...,46,16,317,76,33,154,385,1336,https://www.sports-reference.com/cbb/schools/w...,2018
4,2018,1,"Dallas, TX",Florida,6,77,"Gainesville, Florida",882.710411,St. Bonaventure,11,...,85,80,63,84,120,239,391,1208,https://www.sports-reference.com/cbb/schools/s...,2018


In [19]:
list(outcomes_full_stats.columns.values)

['year',
 'round',
 'location',
 'team_name',
 'team_seed',
 'team_score',
 'team_location',
 'team_travel_miles',
 'opponent_name',
 'opponent_seed',
 'opponent_score',
 'opponent_location',
 'opponent_travel_miles',
 'outcome',
 'name',
 'opp_g',
 'opp_mp',
 'opp_opp_ast',
 'opp_opp_blk',
 'opp_opp_drb',
 'opp_opp_fg',
 'opp_opp_fg2',
 'opp_opp_fg2_pct',
 'opp_opp_fg2a',
 'opp_opp_fg3',
 'opp_opp_fg3_pct',
 'opp_opp_fg3a',
 'opp_opp_fg_pct',
 'opp_opp_fga',
 'opp_opp_ft',
 'opp_opp_ft_pct',
 'opp_opp_fta',
 'opp_opp_orb',
 'opp_opp_pf',
 'opp_opp_pts',
 'opp_opp_pts_per_g',
 'opp_opp_stl',
 'opp_opp_tov',
 'opp_opp_trb',
 'opp_rank_g',
 'opp_rank_mp',
 'opp_rank_opp_ast',
 'opp_rank_opp_blk',
 'opp_rank_opp_drb',
 'opp_rank_opp_fg',
 'opp_rank_opp_fg2',
 'opp_rank_opp_fg2_pct',
 'opp_rank_opp_fg2a',
 'opp_rank_opp_fg3',
 'opp_rank_opp_fg3_pct',
 'opp_rank_opp_fg3a',
 'opp_rank_opp_fg_pct',
 'opp_rank_opp_fga',
 'opp_rank_opp_ft',
 'opp_rank_opp_ft_pct',
 'opp_rank_opp_fta',
 'opp_ran

In [20]:
model_input_df = outcomes_full_stats[['year', 'round', 'location', 'outcome', 'team_name', 'team_seed', 'team_score', 
                                     'team_location', 'team_travel_miles', 'opponent_name', 'opponent_seed', 
                                     'opponent_score', 'opponent_location', 'opponent_travel_miles', 'team_SOS', 
                                     'team_SRS', 'team_ast', 'team_blk', 'team_drb', 'team_fg', 'team_fg2',
                                      'team_fg2_pct', 'team_fg2a', 'team_fg3', 'team_fg3_pct', 'team_fg3a', 
                                      'team_fg_pct', 'team_fga', 'team_ft', 'team_ft_pct', 'team_fta', 'team_g', 
                                      'team_mp', 'team_orb', 'team_pf', 'team_pts', 'team_pts_per_g', 'team_rank_SOS', 
                                      'team_rank_SRS', 'team_rank_ast', 'team_rank_blk', 'team_rank_drb', 'team_rank_fg', 
                                      'team_rank_fg2', 'team_rank_fg2_pct', 'team_rank_fg2a', 'team_rank_fg3', 
                                      'team_rank_fg3_pct', 'team_rank_fg3a', 'team_rank_fg_pct', 'team_rank_fga', 
                                      'team_rank_ft', 'team_rank_ft_pct', 'team_rank_fta', 'team_rank_g', 'team_rank_mp', 
                                      'team_rank_orb', 'team_rank_pf', 'team_rank_pts', 'team_rank_pts_per_g', 
                                      'team_rank_stl', 'team_rank_tov', 'team_rank_trb', 'team_stl', 'team_tov', 
                                      'team_trb', 'opp_g', 'opp_mp', 'opp_opp_ast', 'opp_opp_blk', 'opp_opp_drb', 
                                      'opp_opp_fg', 'opp_opp_fg2', 'opp_opp_fg2_pct', 'opp_opp_fg2a', 'opp_opp_fg3', 
                                      'opp_opp_fg3_pct', 'opp_opp_fg3a', 'opp_opp_fg_pct', 'opp_opp_fga', 'opp_opp_ft', 
                                      'opp_opp_ft_pct', 'opp_opp_fta', 'opp_opp_orb', 'opp_opp_pf', 'opp_opp_pts',
                                      'opp_opp_pts_per_g', 'opp_opp_stl', 'opp_opp_tov', 'opp_opp_trb', 'opp_rank_g',
                                      'opp_rank_mp', 'opp_rank_opp_ast', 'opp_rank_opp_blk', 'opp_rank_opp_drb',
                                      'opp_rank_opp_fg', 'opp_rank_opp_fg2', 'opp_rank_opp_fg2_pct', 'opp_rank_opp_fg2a', 
                                      'opp_rank_opp_fg3', 'opp_rank_opp_fg3_pct', 'opp_rank_opp_fg3a','opp_rank_opp_fg_pct', 
                                      'opp_rank_opp_fga', 'opp_rank_opp_ft', 'opp_rank_opp_ft_pct', 'opp_rank_opp_fta',
                                      'opp_rank_opp_orb', 'opp_rank_opp_pf', 'opp_rank_opp_pts', 'opp_rank_opp_pts_per_g',
                                      'opp_rank_opp_stl', 'opp_rank_opp_tov', 'opp_rank_opp_trb',  'opponent_team_SOS', 
                                      'opponent_team_SRS', 'opponent_team_ast', 'opponent_team_blk', 'opponent_team_drb', 
                                      'opponent_team_fg', 'opponent_team_fg2', 'opponent_team_fg2_pct', 'opponent_team_fg2a', 
                                      'opponent_team_fg3', 'opponent_team_fg3_pct', 'opponent_team_fg3a', 'opponent_team_fg_pct', 
                                      'opponent_team_fga', 'opponent_team_ft', 'opponent_team_ft_pct', 'opponent_team_fta', 
                                      'opponent_team_g', 'opponent_team_mp', 'opponent_team_orb', 'opponent_team_pf', 
                                      'opponent_team_pts', 'opponent_team_pts_per_g', 'opponent_team_rank_SOS', 
                                      'opponent_team_rank_SRS', 'opponent_team_rank_ast', 'opponent_team_rank_blk', 
                                      'opponent_team_rank_drb', 'opponent_team_rank_fg', 'opponent_team_rank_fg2', 
                                      'opponent_team_rank_fg2_pct', 'opponent_team_rank_fg2a', 'opponent_team_rank_fg3', 
                                      'opponent_team_rank_fg3_pct', 'opponent_team_rank_fg3a', 'opponent_team_rank_fg_pct', 
                                      'opponent_team_rank_fga', 'opponent_team_rank_ft', 'opponent_team_rank_ft_pct', 
                                      'opponent_team_rank_fta', 'opponent_team_rank_g', 'opponent_team_rank_mp', 
                                      'opponent_team_rank_orb', 'opponent_team_rank_pf', 'opponent_team_rank_pts', 
                                      'opponent_team_rank_pts_per_g', 'opponent_team_rank_stl', 'opponent_team_rank_tov', 
                                      'opponent_team_rank_trb', 'opponent_team_stl', 'opponent_team_tov', 'opponent_team_trb', 
                                      'opponent_opp_g', 'opponent_opp_mp', 'opponent_opp_opp_ast', 'opponent_opp_opp_blk', 
                                      'opponent_opp_opp_drb', 'opponent_opp_opp_fg', 'opponent_opp_opp_fg2', 
                                      'opponent_opp_opp_fg2_pct', 'opponent_opp_opp_fg2a', 'opponent_opp_opp_fg3', 
                                      'opponent_opp_opp_fg3_pct', 'opponent_opp_opp_fg3a', 'opponent_opp_opp_fg_pct', 
                                      'opponent_opp_opp_fga', 'opponent_opp_opp_ft', 'opponent_opp_opp_ft_pct', 
                                      'opponent_opp_opp_fta', 'opponent_opp_opp_orb', 'opponent_opp_opp_pf', 
                                      'opponent_opp_opp_pts', 'opponent_opp_opp_pts_per_g', 'opponent_opp_opp_stl', 
                                      'opponent_opp_opp_tov', 'opponent_opp_opp_trb', 'opponent_opp_rank_g', 
                                      'opponent_opp_rank_mp', 'opponent_opp_rank_opp_ast', 'opponent_opp_rank_opp_blk', 
                                      'opponent_opp_rank_opp_drb', 'opponent_opp_rank_opp_fg', 'opponent_opp_rank_opp_fg2', 
                                      'opponent_opp_rank_opp_fg2_pct', 'opponent_opp_rank_opp_fg2a', 'opponent_opp_rank_opp_fg3', 
                                      'opponent_opp_rank_opp_fg3_pct', 'opponent_opp_rank_opp_fg3a', 'opponent_opp_rank_opp_fg_pct', 
                                      'opponent_opp_rank_opp_fga', 'opponent_opp_rank_opp_ft', 'opponent_opp_rank_opp_ft_pct', 
                                      'opponent_opp_rank_opp_fta', 'opponent_opp_rank_opp_orb', 'opponent_opp_rank_opp_pf', 
                                      'opponent_opp_rank_opp_pts', 'opponent_opp_rank_opp_pts_per_g', 'opponent_opp_rank_opp_stl', 
                                      'opponent_opp_rank_opp_tov', 'opponent_opp_rank_opp_trb']]

pd.set_option('display.max_columns', 500)
model_input_df.head()

,year,round,location,outcome,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,team_SOS,team_SRS,team_ast,team_blk,team_drb,team_fg,team_fg2,team_fg2_pct,team_fg2a,team_fg3,team_fg3_pct,team_fg3a,team_fg_pct,team_fga,team_ft,team_ft_pct,team_fta,team_g,team_mp,team_orb,team_pf,team_pts,team_pts_per_g,team_rank_SOS,team_rank_SRS,team_rank_ast,team_rank_blk,team_rank_drb,team_rank_fg,team_rank_fg2,team_rank_fg2_pct,team_rank_fg2a,team_rank_fg3,team_rank_fg3_pct,team_rank_fg3a,team_rank_fg_pct,team_rank_fga,team_rank_ft,team_rank_ft_pct,team_rank_fta,team_rank_g,team_rank_mp,team_rank_orb,team_rank_pf,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_tov,team_rank_trb,team_stl,team_tov,team_trb,opp_g,opp_mp,opp_opp_ast,opp_opp_blk,opp_opp_drb,opp_opp_fg,opp_opp_fg2,opp_opp_fg2_pct,opp_opp_fg2a,opp_opp_fg3,opp_opp_fg3_pct,opp_opp_fg3a,opp_opp_fg_pct,opp_opp_fga,opp_opp_ft,opp_opp_ft_pct,opp_opp_fta,opp_opp_orb,opp_opp_pf,opp_opp_pts,opp_opp_pts_per_g,opp_opp_stl,opp_opp_tov,opp_opp_trb,opp_rank_g,opp_rank_mp,opp_rank_opp_ast,opp_rank_opp_blk,opp_rank_opp_drb,opp_rank_opp_fg,opp_rank_opp_fg2,opp_rank_opp_fg2_pct,opp_rank_opp_fg2a,opp_rank_opp_fg3,opp_rank_opp_fg3_pct,opp_rank_opp_fg3a,opp_rank_opp_fg_pct,opp_rank_opp_fga,opp_rank_opp_ft,opp_rank_opp_ft_pct,opp_rank_opp_fta,opp_rank_opp_orb,opp_rank_opp_pf,opp_rank_opp_pts,opp_rank_opp_pts_per_g,opp_rank_opp_stl,opp_rank_opp_tov,opp_rank_opp_trb,opponent_team_SOS,opponent_team_SRS,opponent_team_ast,opponent_team_blk,opponent_team_drb,opponent_team_fg,opponent_team_fg2,opponent_team_fg2_pct,opponent_team_fg2a,opponent_team_fg3,opponent_team_fg3_pct,opponent_team_fg3a,opponent_team_fg_pct,opponent_team_fga,opponent_team_ft,opponent_team_ft_pct,opponent_team_fta,opponent_team_g,opponent_team_mp,opponent_team_orb,opponent_team_pf,opponent_team_pts,opponent_team_pts_per_g,opponent_team_rank_SOS,opponent_team_rank_SRS,opponent_team_rank_ast,opponent_team_rank_blk,opponent_team_rank_drb,opponent_team_rank_fg,opponent_team_rank_fg2,opponent_team_rank_fg2_pct,opponent_team_rank_fg2a,opponent_team_rank_fg3,opponent_team_rank_fg3_pct,opponent_team_rank_fg3a,opponent_team_rank_fg_pct,opponent_team_rank_fga,opponent_team_rank_ft,opponent_team_rank_ft_pct,opponent_team_rank_fta,opponent_team_rank_g,opponent_team_rank_mp,opponent_team_rank_orb,opponent_team_rank_pf,opponent_team_rank_pts,opponent_team_rank_pts_per_g,opponent_team_rank_stl,opponent_team_rank_tov,opponent_team_rank_trb,opponent_team_stl,opponent_team_tov,opponent_team_trb,opponent_opp_g,opponent_opp_mp,opponent_opp_opp_ast,opponent_opp_opp_blk,opponent_opp_opp_drb,opponent_opp_opp_fg,opponent_opp_opp_fg2,opponent_opp_opp_fg2_pct,opponent_opp_opp_fg2a,opponent_opp_opp_fg3,opponent_opp_opp_fg3_pct,opponent_opp_opp_fg3a,opponent_opp_opp_fg_pct,opponent_opp_opp_fga,opponent_opp_opp_ft,opponent_opp_opp_ft_pct,opponent_opp_opp_fta,opponent_opp_opp_orb,opponent_opp_opp_pf,opponent_opp_opp_pts,opponent_opp_opp_pts_per_g,opponent_opp_opp_stl,opponent_opp_opp_tov,opponent_opp_opp_trb,opponent_opp_rank_g,opponent_opp_rank_mp,opponent_opp_rank_opp_ast,opponent_opp_rank_opp_blk,opponent_opp_rank_opp_drb,opponent_opp_rank_opp_fg,opponent_opp_rank_opp_fg2,opponent_opp_rank_opp_fg2_pct,opponent_opp_rank_opp_fg2a,opponent_opp_rank_opp_fg3,opponent_opp_rank_opp_fg3_pct,opponent_opp_rank_opp_fg3a,opponent_opp_rank_opp_fg_pct,opponent_opp_rank_opp_fga,opponent_opp_rank_opp_ft,opponent_opp_rank_opp_ft_pct,opponent_opp_rank_opp_fta,opponent_opp_rank_opp_orb,opponent_opp_rank_opp_pf,opponent_opp_rank_opp_pts,opponent_opp_rank_opp_pts_per_g,opponent_opp_rank_opp_stl,opponent_opp_rank_opp_tov,opponent_opp_rank_opp_trb
0,2018,1,"Pittsburgh, PA",WIN,Villanova,1,87,"Villanova, Pennsylvania",246.946684,Radford,16,61,"Radford, Virginia",230.496683,10.24,26.64,655,162,1056,1220,756,0.590,1282,464,0.401,1158,0.500,2440,559,0.779,718,40,8075,380,645,3463,86.6,10,

## Create/Train/Test/Refine Model

In [50]:
X = model_input_df[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_SOS', 'team_SRS', 
                    'opponent_team_SOS', 'opponent_team_SRS']]
y = model_input_df['outcome']

print(X.shape, y.shape)

(3744, 8) (3744,)


In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (X, y, random_state=42, stratify=y)

In [23]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
classifier.score(X_test, y_test)

0.7403846153846154

In [25]:
predictions = classifier.predict(X_test)

In [26]:
probabilities = classifier.predict_proba(X_test)

In [27]:
loss_prob = []
win_prob = []

for prob in probabilities:
    loss_prob.append(prob[0])
    win_prob.append(prob[1])

In [28]:
upsets = model_input_df.loc[((model_input_df['team_seed'] > model_input_df['opponent_seed'])
                            & (model_input_df['outcome'] == 'WIN')) | ((model_input_df['team_seed'] < model_input_df['opponent_seed'])
                            & (model_input_df['outcome'] == 'LOSS')),:]
upsets

,year,round,location,outcome,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,team_SOS,team_SRS,team_ast,team_blk,team_drb,team_fg,team_fg2,team_fg2_pct,team_fg2a,team_fg3,team_fg3_pct,team_fg3a,team_fg_pct,team_fga,team_ft,team_ft_pct,team_fta,team_g,team_mp,team_orb,team_pf,team_pts,team_pts_per_g,team_rank_SOS,team_rank_SRS,team_rank_ast,team_rank_blk,team_rank_drb,team_rank_fg,team_rank_fg2,team_rank_fg2_pct,team_rank_fg2a,team_rank_fg3,team_rank_fg3_pct,team_rank_fg3a,team_rank_fg_pct,team_rank_fga,team_rank_ft,team_rank_ft_pct,team_rank_fta,team_rank_g,team_rank_mp,team_rank_orb,team_rank_pf,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_tov,team_rank_trb,team_stl,team_tov,team_trb,opp_g,opp_mp,opp_opp_ast,opp_opp_blk,opp_opp_drb,opp_opp_fg,opp_opp_fg2,opp_opp_fg2_pct,opp_opp_fg2a,opp_opp_fg3,opp_opp_fg3_pct,opp_opp_fg3a,opp_opp_fg_pct,opp_opp_fga,opp_opp_ft,opp_opp_ft_pct,opp_opp_fta,opp_opp_orb,opp_opp_pf,opp_opp_pts,opp_opp_pts_per_g,opp_opp_stl,opp_opp_tov,opp_opp_trb,opp_rank_g,opp_rank_mp,opp_rank_opp_ast,opp_rank_opp_blk,opp_rank_opp_drb,opp_rank_opp_fg,opp_rank_opp_fg2,opp_rank_opp_fg2_pct,opp_rank_opp_fg2a,opp_rank_opp_fg3,opp_rank_opp_fg3_pct,opp_rank_opp_fg3a,opp_rank_opp_fg_pct,opp_rank_opp_fga,opp_rank_opp_ft,opp_rank_opp_ft_pct,opp_rank_opp_fta,opp_rank_opp_orb,opp_rank_opp_pf,opp_rank_opp_pts,opp_rank_opp_pts_per_g,opp_rank_opp_stl,opp_rank_opp_tov,opp_rank_opp_trb,opponent_team_SOS,opponent_team_SRS,opponent_team_ast,opponent_team_blk,opponent_team_drb,opponent_team_fg,opponent_team_fg2,opponent_team_fg2_pct,opponent_team_fg2a,opponent_team_fg3,opponent_team_fg3_pct,opponent_team_fg3a,opponent_team_fg_pct,opponent_team_fga,opponent_team_ft,opponent_team_ft_pct,opponent_team_fta,opponent_team_g,opponent_team_mp,opponent_team_orb,opponent_team_pf,opponent_team_pts,opponent_team_pts_per_g,opponent_team_rank_SOS,opponent_team_rank_SRS,opponent_team_rank_ast,opponent_team_rank_blk,opponent_team_rank_drb,opponent_team_rank_fg,opponent_team_rank_fg2,opponent_team_rank_fg2_pct,opponent_team_rank_fg2a,opponent_team_rank_fg3,opponent_team_rank_fg3_pct,opponent_team_rank_fg3a,opponent_team_rank_fg_pct,opponent_team_rank_fga,opponent_team_rank_ft,opponent_team_rank_ft_pct,opponent_team_rank_fta,opponent_team_rank_g,opponent_team_rank_mp,opponent_team_rank_orb,opponent_team_rank_pf,opponent_team_rank_pts,opponent_team_rank_pts_per_g,opponent_team_rank_stl,opponent_team_rank_tov,opponent_team_rank_trb,opponent_team_stl,opponent_team_tov,opponent_team_trb,opponent_opp_g,opponent_opp_mp,opponent_opp_opp_ast,opponent_opp_opp_blk,opponent_opp_opp_drb,opponent_opp_opp_fg,opponent_opp_opp_fg2,opponent_opp_opp_fg2_pct,opponent_opp_opp_fg2a,opponent_opp_opp_fg3,opponent_opp_opp_fg3_pct,opponent_opp_opp_fg3a,opponent_opp_opp_fg_pct,opponent_opp_opp_fga,opponent_opp_opp_ft,opponent_opp_opp_ft_pct,opponent_opp_opp_fta,opponent_opp_opp_orb,opponent_opp_opp_pf,opponent_opp_opp_pts,opponent_opp_opp_pts_per_g,opponent_opp_opp_stl,opponent_opp_opp_tov,opponent_opp_opp_trb,opponent_opp_rank_g,opponent_opp_rank_mp,opponent_opp_rank_opp_ast,opponent_opp_rank_opp_blk,opponent_opp_rank_opp_drb,opponent_opp_rank_opp_fg,opponent_opp_rank_opp_fg2,opponent_opp_rank_opp_fg2_pct,opponent_opp_rank_opp_fg2a,opponent_opp_rank_opp_fg3,opponent_opp_rank_opp_fg3_pct,opponent_opp_rank_opp_fg3a,opponent_opp_rank_opp_fg_pct,opponent_opp_rank_opp_fga,opponent_opp_rank_opp_ft,opponent_opp_rank_opp_ft_pct,opponent_opp_rank_opp_fta,opponent_opp_rank_opp_orb,opponent_opp_rank_opp_pf,opponent_opp_rank_opp_pts,opponent_opp_rank_opp_pts_per_g,opponent_opp_rank_opp_stl,opponent_opp_rank_opp_tov,opponent_opp_rank_opp_trb
1,2018,1,"Pittsburgh, PA",WIN,Alabama,9,86,"Tuscaloosa, Alabama",645.478758,Virginia Tech,8,83,"Blacksburg, Virginia",222.738124,10.64,12.34,458,192,932,910,681,0.535,1273,229,0.326,702,0.461,1975,555,0.670,828,36,7200,358,676,2604,72.3,6,49

In [29]:
X_upsets = upsets[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_SOS', 'team_SRS', 
                    'opponent_team_SOS', 'opponent_team_SRS']]
y_upsets = upsets['outcome']

In [30]:
classifier.score(X_upsets, y_upsets)

0.3058350100603622

In [31]:
rd_1_upsets = model_input_df.loc[((model_input_df['team_seed'] > model_input_df['opponent_seed'])
                            & (model_input_df['outcome'] == 'WIN')) | ((model_input_df['team_seed'] < model_input_df['opponent_seed'])
                            & (model_input_df['outcome'] == 'LOSS')) & (model_input_df['round'] == 1),:]
rd_1_upsets

,year,round,location,outcome,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,team_SOS,team_SRS,team_ast,team_blk,team_drb,team_fg,team_fg2,team_fg2_pct,team_fg2a,team_fg3,team_fg3_pct,team_fg3a,team_fg_pct,team_fga,team_ft,team_ft_pct,team_fta,team_g,team_mp,team_orb,team_pf,team_pts,team_pts_per_g,team_rank_SOS,team_rank_SRS,team_rank_ast,team_rank_blk,team_rank_drb,team_rank_fg,team_rank_fg2,team_rank_fg2_pct,team_rank_fg2a,team_rank_fg3,team_rank_fg3_pct,team_rank_fg3a,team_rank_fg_pct,team_rank_fga,team_rank_ft,team_rank_ft_pct,team_rank_fta,team_rank_g,team_rank_mp,team_rank_orb,team_rank_pf,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_tov,team_rank_trb,team_stl,team_tov,team_trb,opp_g,opp_mp,opp_opp_ast,opp_opp_blk,opp_opp_drb,opp_opp_fg,opp_opp_fg2,opp_opp_fg2_pct,opp_opp_fg2a,opp_opp_fg3,opp_opp_fg3_pct,opp_opp_fg3a,opp_opp_fg_pct,opp_opp_fga,opp_opp_ft,opp_opp_ft_pct,opp_opp_fta,opp_opp_orb,opp_opp_pf,opp_opp_pts,opp_opp_pts_per_g,opp_opp_stl,opp_opp_tov,opp_opp_trb,opp_rank_g,opp_rank_mp,opp_rank_opp_ast,opp_rank_opp_blk,opp_rank_opp_drb,opp_rank_opp_fg,opp_rank_opp_fg2,opp_rank_opp_fg2_pct,opp_rank_opp_fg2a,opp_rank_opp_fg3,opp_rank_opp_fg3_pct,opp_rank_opp_fg3a,opp_rank_opp_fg_pct,opp_rank_opp_fga,opp_rank_opp_ft,opp_rank_opp_ft_pct,opp_rank_opp_fta,opp_rank_opp_orb,opp_rank_opp_pf,opp_rank_opp_pts,opp_rank_opp_pts_per_g,opp_rank_opp_stl,opp_rank_opp_tov,opp_rank_opp_trb,opponent_team_SOS,opponent_team_SRS,opponent_team_ast,opponent_team_blk,opponent_team_drb,opponent_team_fg,opponent_team_fg2,opponent_team_fg2_pct,opponent_team_fg2a,opponent_team_fg3,opponent_team_fg3_pct,opponent_team_fg3a,opponent_team_fg_pct,opponent_team_fga,opponent_team_ft,opponent_team_ft_pct,opponent_team_fta,opponent_team_g,opponent_team_mp,opponent_team_orb,opponent_team_pf,opponent_team_pts,opponent_team_pts_per_g,opponent_team_rank_SOS,opponent_team_rank_SRS,opponent_team_rank_ast,opponent_team_rank_blk,opponent_team_rank_drb,opponent_team_rank_fg,opponent_team_rank_fg2,opponent_team_rank_fg2_pct,opponent_team_rank_fg2a,opponent_team_rank_fg3,opponent_team_rank_fg3_pct,opponent_team_rank_fg3a,opponent_team_rank_fg_pct,opponent_team_rank_fga,opponent_team_rank_ft,opponent_team_rank_ft_pct,opponent_team_rank_fta,opponent_team_rank_g,opponent_team_rank_mp,opponent_team_rank_orb,opponent_team_rank_pf,opponent_team_rank_pts,opponent_team_rank_pts_per_g,opponent_team_rank_stl,opponent_team_rank_tov,opponent_team_rank_trb,opponent_team_stl,opponent_team_tov,opponent_team_trb,opponent_opp_g,opponent_opp_mp,opponent_opp_opp_ast,opponent_opp_opp_blk,opponent_opp_opp_drb,opponent_opp_opp_fg,opponent_opp_opp_fg2,opponent_opp_opp_fg2_pct,opponent_opp_opp_fg2a,opponent_opp_opp_fg3,opponent_opp_opp_fg3_pct,opponent_opp_opp_fg3a,opponent_opp_opp_fg_pct,opponent_opp_opp_fga,opponent_opp_opp_ft,opponent_opp_opp_ft_pct,opponent_opp_opp_fta,opponent_opp_opp_orb,opponent_opp_opp_pf,opponent_opp_opp_pts,opponent_opp_opp_pts_per_g,opponent_opp_opp_stl,opponent_opp_opp_tov,opponent_opp_opp_trb,opponent_opp_rank_g,opponent_opp_rank_mp,opponent_opp_rank_opp_ast,opponent_opp_rank_opp_blk,opponent_opp_rank_opp_drb,opponent_opp_rank_opp_fg,opponent_opp_rank_opp_fg2,opponent_opp_rank_opp_fg2_pct,opponent_opp_rank_opp_fg2a,opponent_opp_rank_opp_fg3,opponent_opp_rank_opp_fg3_pct,opponent_opp_rank_opp_fg3a,opponent_opp_rank_opp_fg_pct,opponent_opp_rank_opp_fga,opponent_opp_rank_opp_ft,opponent_opp_rank_opp_ft_pct,opponent_opp_rank_opp_fta,opponent_opp_rank_opp_orb,opponent_opp_rank_opp_pf,opponent_opp_rank_opp_pts,opponent_opp_rank_opp_pts_per_g,opponent_opp_rank_opp_stl,opponent_opp_rank_opp_tov,opponent_opp_rank_opp_trb
1,2018,1,"Pittsburgh, PA",WIN,Alabama,9,86,"Tuscaloosa, Alabama",645.478758,Virginia Tech,8,83,"Blacksburg, Virginia",222.738124,10.64,12.34,458,192,932,910,681,0.535,1273,229,0.326,702,0.461,1975,555,0.670,828,36,7200,358,676,2604,72.3,6,49

In [32]:
X_upsets_rd_1 = rd_1_upsets[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_SOS', 'team_SRS', 
                    'opponent_team_SOS', 'opponent_team_SRS']]
y_upsets_rd_1 = rd_1_upsets['outcome']

In [33]:
classifier.score(X_upsets_rd_1, y_upsets_rd_1)

0.3112676056338028

In [34]:
X_seed_only = model_input_df[['team_seed', 'opponent_seed']]
y_seed_only = model_input_df['outcome']

In [69]:
X_train_seed_only, X_test_seed_only, y_train_seed_only, y_test_seed_only = train_test_split(X_seed_only, y_seed_only, random_state=42, stratify=y_seed_only)

In [70]:
classifier_seed_only = LogisticRegression()
classifier_seed_only.fit(X_train_seed_only, y_train_seed_only)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [71]:
classifier_seed_only.score(X_test_seed_only, y_test_seed_only)

0.7104700854700855

In [72]:
X_upsets_seed_only = upsets[['team_seed', 'opponent_seed']]
y_upsets_seed_only = upsets['outcome']

In [73]:
classifier_seed_only.score(X_upsets_seed_only, y_upsets_seed_only)

0.0

In [40]:
model_input_df['year'].min()

'1993'

In [52]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [53]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [55]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [56]:
classifier.score(X_test_scaled, y_test)

0.7393162393162394

In [57]:
X_upsets = upsets[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_SOS', 'team_SRS', 
                    'opponent_team_SOS', 'opponent_team_SRS']]
y_upsets = upsets['outcome']

In [58]:
X_upsets_scaled = X_scaler.transform(X_upsets)

In [59]:
classifier.score(X_upsets_scaled, y_upsets)

0.3038229376257545

In [67]:
tourns_after_2000 = model_input_df.loc[(model_input_df['year'] == '2000') | (model_input_df['year'] == '2001') |
                                       (model_input_df['year'] == '2002') | (model_input_df['year'] == '2003') |
                                       (model_input_df['year'] == '2004') | (model_input_df['year'] == '2005') |
                                       (model_input_df['year'] == '2006') | (model_input_df['year'] == '2007') |
                                       (model_input_df['year'] == '2008') | (model_input_df['year'] == '2009') |
                                       (model_input_df['year'] == '2010') | (model_input_df['year'] == '2011') |
                                       (model_input_df['year'] == '2012') | (model_input_df['year'] == '2013') |
                                       (model_input_df['year'] == '2014') | (model_input_df['year'] == '2015') |
                                       (model_input_df['year'] == '2016') | (model_input_df['year'] == '2017') |
                                       (model_input_df['year'] == '2018'), :]
len(tourns_after_2000)

2736

In [68]:
X_2000 = tourns_after_2000[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_SOS', 'team_SRS', 
                    'opponent_team_SOS', 'opponent_team_SRS']]
y_2000 = tourns_after_2000['outcome']

In [74]:
X_train_2000, X_test_2000, y_train_2000, y_test_2000 = train_test_split(X_2000, y_2000, random_state=42, stratify=y_2000)


In [75]:
classifier_2000 = LogisticRegression()

In [76]:
classifier.fit(X_train_2000, y_train_2000)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [77]:
classifier.score(X_test_2000, y_test_2000)

0.7587719298245614

In [78]:
upsets_after_2000 = upsets.loc[(upsets['year'] == '2000') | (upsets['year'] == '2001') |
                                       (upsets['year'] == '2002') | (upsets['year'] == '2003') |
                                       (upsets['year'] == '2004') | (upsets['year'] == '2005') |
                                       (upsets['year'] == '2006') | (upsets['year'] == '2007') |
                                       (upsets['year'] == '2008') | (upsets['year'] == '2009') |
                                       (upsets['year'] == '2010') | (upsets['year'] == '2011') |
                                       (upsets['year'] == '2012') | (upsets['year'] == '2013') |
                                       (upsets['year'] == '2014') | (upsets['year'] == '2015') |
                                       (upsets['year'] == '2016') | (upsets['year'] == '2017') |
                                       (upsets['year'] == '2018'), :]
len(upsets)

994

In [79]:
X_upsets_2000 = upsets_after_2000[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 
                                   'team_SOS', 'team_SRS', 'opponent_team_SOS', 'opponent_team_SRS']]
y_upsets_2000 = upsets_after_2000['outcome']

In [80]:
classifier.score(X_upsets_2000, y_upsets_2000)

0.3293010752688172

In [81]:
X = tourns_after_2000[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_SRS', 
                       'opponent_team_SRS']]
y = tourns_after_2000['outcome']

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [83]:
classifier = LogisticRegression()

In [84]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [85]:
classifier.score(X_test, y_test)

0.7602339181286549

In [86]:
X_upsets = upsets_after_2000[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_SRS', 
                       'opponent_team_SRS']]
y_upsets = upsets_after_2000['outcome']

In [87]:
classifier.score(X_upsets, y_upsets)

0.3293010752688172

In [88]:
X = tourns_after_2000[['team_travel_miles', 'opponent_travel_miles', 'team_SRS', 'opponent_team_SRS']]
y = tourns_after_2000['outcome']

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [90]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [91]:
classifier.score(X_test, y_test)

0.7573099415204678

In [92]:
X_upsets = upsets_after_2000[['team_travel_miles', 'opponent_travel_miles', 'team_SRS', 'opponent_team_SRS']]
y_upsets = upsets_after_2000['outcome']

In [93]:
classifier.score(X_upsets, y_upsets)

0.25

In [96]:
tourns_after_2000.head(1)

,year,round,location,outcome,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,team_SOS,team_SRS,team_ast,team_blk,team_drb,team_fg,team_fg2,team_fg2_pct,team_fg2a,team_fg3,team_fg3_pct,team_fg3a,team_fg_pct,team_fga,team_ft,team_ft_pct,team_fta,team_g,team_mp,team_orb,team_pf,team_pts,team_pts_per_g,team_rank_SOS,team_rank_SRS,team_rank_ast,team_rank_blk,team_rank_drb,team_rank_fg,team_rank_fg2,team_rank_fg2_pct,team_rank_fg2a,team_rank_fg3,team_rank_fg3_pct,team_rank_fg3a,team_rank_fg_pct,team_rank_fga,team_rank_ft,team_rank_ft_pct,team_rank_fta,team_rank_g,team_rank_mp,team_rank_orb,team_rank_pf,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_tov,team_rank_trb,team_stl,team_tov,team_trb,opp_g,opp_mp,opp_opp_ast,opp_opp_blk,opp_opp_drb,opp_opp_fg,opp_opp_fg2,opp_opp_fg2_pct,opp_opp_fg2a,opp_opp_fg3,opp_opp_fg3_pct,opp_opp_fg3a,opp_opp_fg_pct,opp_opp_fga,opp_opp_ft,opp_opp_ft_pct,opp_opp_fta,opp_opp_orb,opp_opp_pf,opp_opp_pts,opp_opp_pts_per_g,opp_opp_stl,opp_opp_tov,opp_opp_trb,opp_rank_g,opp_rank_mp,opp_rank_opp_ast,opp_rank_opp_blk,opp_rank_opp_drb,opp_rank_opp_fg,opp_rank_opp_fg2,opp_rank_opp_fg2_pct,opp_rank_opp_fg2a,opp_rank_opp_fg3,opp_rank_opp_fg3_pct,opp_rank_opp_fg3a,opp_rank_opp_fg_pct,opp_rank_opp_fga,opp_rank_opp_ft,opp_rank_opp_ft_pct,opp_rank_opp_fta,opp_rank_opp_orb,opp_rank_opp_pf,opp_rank_opp_pts,opp_rank_opp_pts_per_g,opp_rank_opp_stl,opp_rank_opp_tov,opp_rank_opp_trb,opponent_team_SOS,opponent_team_SRS,opponent_team_ast,opponent_team_blk,opponent_team_drb,opponent_team_fg,opponent_team_fg2,opponent_team_fg2_pct,opponent_team_fg2a,opponent_team_fg3,opponent_team_fg3_pct,opponent_team_fg3a,opponent_team_fg_pct,opponent_team_fga,opponent_team_ft,opponent_team_ft_pct,opponent_team_fta,opponent_team_g,opponent_team_mp,opponent_team_orb,opponent_team_pf,opponent_team_pts,opponent_team_pts_per_g,opponent_team_rank_SOS,opponent_team_rank_SRS,opponent_team_rank_ast,opponent_team_rank_blk,opponent_team_rank_drb,opponent_team_rank_fg,opponent_team_rank_fg2,opponent_team_rank_fg2_pct,opponent_team_rank_fg2a,opponent_team_rank_fg3,opponent_team_rank_fg3_pct,opponent_team_rank_fg3a,opponent_team_rank_fg_pct,opponent_team_rank_fga,opponent_team_rank_ft,opponent_team_rank_ft_pct,opponent_team_rank_fta,opponent_team_rank_g,opponent_team_rank_mp,opponent_team_rank_orb,opponent_team_rank_pf,opponent_team_rank_pts,opponent_team_rank_pts_per_g,opponent_team_rank_stl,opponent_team_rank_tov,opponent_team_rank_trb,opponent_team_stl,opponent_team_tov,opponent_team_trb,opponent_opp_g,opponent_opp_mp,opponent_opp_opp_ast,opponent_opp_opp_blk,opponent_opp_opp_drb,opponent_opp_opp_fg,opponent_opp_opp_fg2,opponent_opp_opp_fg2_pct,opponent_opp_opp_fg2a,opponent_opp_opp_fg3,opponent_opp_opp_fg3_pct,opponent_opp_opp_fg3a,opponent_opp_opp_fg_pct,opponent_opp_opp_fga,opponent_opp_opp_ft,opponent_opp_opp_ft_pct,opponent_opp_opp_fta,opponent_opp_opp_orb,opponent_opp_opp_pf,opponent_opp_opp_pts,opponent_opp_opp_pts_per_g,opponent_opp_opp_stl,opponent_opp_opp_tov,opponent_opp_opp_trb,opponent_opp_rank_g,opponent_opp_rank_mp,opponent_opp_rank_opp_ast,opponent_opp_rank_opp_blk,opponent_opp_rank_opp_drb,opponent_opp_rank_opp_fg,opponent_opp_rank_opp_fg2,opponent_opp_rank_opp_fg2_pct,opponent_opp_rank_opp_fg2a,opponent_opp_rank_opp_fg3,opponent_opp_rank_opp_fg3_pct,opponent_opp_rank_opp_fg3a,opponent_opp_rank_opp_fg_pct,opponent_opp_rank_opp_fga,opponent_opp_rank_opp_ft,opponent_opp_rank_opp_ft_pct,opponent_opp_rank_opp_fta,opponent_opp_rank_opp_orb,opponent_opp_rank_opp_pf,opponent_opp_rank_opp_pts,opponent_opp_rank_opp_pts_per_g,opponent_opp_rank_opp_stl,opponent_opp_rank_opp_tov,opponent_opp_rank_opp_trb
0,2018,1,"Pittsburgh, PA",WIN,Villanova,1,87,"Villanova, Pennsylvania",246.946684,Radford,16,61,"Radford, Virginia",230.496683,10.24,26.64,655,162,1056,1220,756,0.59,1282,464,0.401,1158,0.5,2440,559,0.779,718,40,8075,380,645,3463,86.6,10,1,3

In [113]:
X = tourns_after_2000[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_rank_SOS', 
                       'team_rank_SRS', 'team_rank_ast', 'team_rank_blk', 'team_rank_fg', 
                      'team_rank_fg2', 'team_rank_fg2_pct', 'team_rank_fg2a', 'team_rank_fg3', 'team_rank_fg3_pct', 
                      'team_rank_fg3a', 'team_rank_fg_pct', 'team_rank_fga', 'team_rank_ft', 'team_rank_ft_pct', 
                      'team_rank_fta', 'team_rank_pts', 'team_rank_pts_per_g', 'team_rank_stl', 'team_rank_trb', 
                       'opponent_team_rank_SOS', 'opponent_team_rank_SRS', 'opponent_team_rank_ast', 
                       'opponent_team_rank_blk', 'opponent_team_rank_fg', 'opponent_team_rank_fg2', 
                       'opponent_team_rank_fg2_pct', 'opponent_team_rank_fg2a', 'opponent_team_rank_fg3', 
                       'opponent_team_rank_fg3_pct', 'opponent_team_rank_fg3a', 'opponent_team_rank_fg_pct', 
                       'opponent_team_rank_fga', 'opponent_team_rank_ft', 'opponent_team_rank_ft_pct', 
                       'opponent_team_rank_fta', 'opponent_team_rank_pts', 'opponent_team_rank_pts_per_g', 
                       'opponent_team_rank_stl', 'opponent_team_rank_trb']]
y = tourns_after_2000['outcome']

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [115]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [116]:
classifier.score(X_test, y_test)

0.8026315789473685

In [117]:
X_upsets = upsets_after_2000[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_rank_SOS', 
                       'team_rank_SRS', 'team_rank_ast', 'team_rank_blk', 'team_rank_fg', 
                      'team_rank_fg2', 'team_rank_fg2_pct', 'team_rank_fg2a', 'team_rank_fg3', 'team_rank_fg3_pct', 
                      'team_rank_fg3a', 'team_rank_fg_pct', 'team_rank_fga', 'team_rank_ft', 'team_rank_ft_pct', 
                      'team_rank_fta', 'team_rank_pts', 'team_rank_pts_per_g', 'team_rank_stl', 'team_rank_trb', 
                       'opponent_team_rank_SOS', 'opponent_team_rank_SRS', 'opponent_team_rank_ast', 
                       'opponent_team_rank_blk', 'opponent_team_rank_fg', 'opponent_team_rank_fg2', 
                       'opponent_team_rank_fg2_pct', 'opponent_team_rank_fg2a', 'opponent_team_rank_fg3', 
                       'opponent_team_rank_fg3_pct', 'opponent_team_rank_fg3a', 'opponent_team_rank_fg_pct', 
                       'opponent_team_rank_fga', 'opponent_team_rank_ft', 'opponent_team_rank_ft_pct', 
                       'opponent_team_rank_fta', 'opponent_team_rank_pts', 'opponent_team_rank_pts_per_g', 
                       'opponent_team_rank_stl', 'opponent_team_rank_trb']]
y_upsets = upsets_after_2000['outcome']

In [118]:
classifier.score(X_upsets, y_upsets)

0.5981182795698925

In [130]:
tourn_2018 = model_input_df.loc[(model_input_df['year'] == '2018') & (model_input_df['round'] == 1), :]
len(tourn_2018)

64

In [131]:
X_2018 = tourn_2018[['team_seed', 'team_travel_miles', 'opponent_seed', 'opponent_travel_miles', 'team_rank_SOS', 
                       'team_rank_SRS', 'team_rank_ast', 'team_rank_blk', 'team_rank_fg', 
                      'team_rank_fg2', 'team_rank_fg2_pct', 'team_rank_fg2a', 'team_rank_fg3', 'team_rank_fg3_pct', 
                      'team_rank_fg3a', 'team_rank_fg_pct', 'team_rank_fga', 'team_rank_ft', 'team_rank_ft_pct', 
                      'team_rank_fta', 'team_rank_pts', 'team_rank_pts_per_g', 'team_rank_stl', 'team_rank_trb', 
                       'opponent_team_rank_SOS', 'opponent_team_rank_SRS', 'opponent_team_rank_ast', 
                       'opponent_team_rank_blk', 'opponent_team_rank_fg', 'opponent_team_rank_fg2', 
                       'opponent_team_rank_fg2_pct', 'opponent_team_rank_fg2a', 'opponent_team_rank_fg3', 
                       'opponent_team_rank_fg3_pct', 'opponent_team_rank_fg3a', 'opponent_team_rank_fg_pct', 
                       'opponent_team_rank_fga', 'opponent_team_rank_ft', 'opponent_team_rank_ft_pct', 
                       'opponent_team_rank_fta', 'opponent_team_rank_pts', 'opponent_team_rank_pts_per_g', 
                       'opponent_team_rank_stl', 'opponent_team_rank_trb']]
y_2018 = tourn_2018['outcome']

In [132]:
classifier.score(X_2018, y_2018)

0.859375

In [133]:
predictions = classifier.predict(X_2018)
probabilities = classifier.predict_proba(X_2018)

In [140]:
import pickle

model_filename = 'ncaa_model.pkl'
with open(model_filename, 'wb') as file:  
    pickle.dump(classifier, file)

In [141]:
with open(model_filename, 'rb') as file:  
    restored_model = pickle.load(file)

In [142]:
restored_model.score(X_2018, y_2018)

0.859375

In [159]:
X_2018_test_game = X_2018.iloc[0:1,:]
y_2018_test_game = y_2018.iloc[0]

X_2018_test_game

,team_seed,team_travel_miles,opponent_seed,opponent_travel_miles,team_rank_SOS,team_rank_SRS,team_rank_ast,team_rank_blk,team_rank_fg,team_rank_fg2,team_rank_fg2_pct,team_rank_fg2a,team_rank_fg3,team_rank_fg3_pct,team_rank_fg3a,team_rank_fg_pct,team_rank_fga,team_rank_ft,team_rank_ft_pct,team_rank_fta,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_trb,opponent_team_rank_SOS,opponent_team_rank_SRS,opponent_team_rank_ast,opponent_team_rank_blk,opponent_team_rank_fg,opponent_team_rank_fg2,opponent_team_rank_fg2_pct,opponent_team_rank_fg2a,opponent_team_rank_fg3,opponent_team_rank_fg3_pct,opponent_team_rank_fg3a,opponent_team_rank_fg_pct,opponent_team_rank_fga,opponent_team_rank_ft,opponent_team_rank_ft_pct,opponent_team_rank_fta,opponent_team_rank_pts,opponent_team_rank_pts_per_g,opponent_team_rank_stl,opponent_team_rank_trb
0,1,246.946684,16,230.496683,10,1,3,31,1,16,4,96,1,13,2,5,1,32,10,73,1,1,28,8,265,191,185,184,175,219,287,167,106,178,87,285,106,199,132,214,169,312,97,93


In [160]:
testing_probability = restored_model.predict_proba(X_2018_test_game)

In [161]:
testing_probability

array([[0.00220883, 0.99779117]])

In [162]:
restored_model.decision_function(X_2018_test_game)

array([6.1130788])